# DATA PROCESSING
(Convert all data in xlfiles/ to db.json, used as the website database)

Output from find_aas.py is referred to as "aa data" and from franken_pas.m as "franken data"
Each bullet point corresponds to a single cell, in order
- convert .xlsx files in xlfiles/ to .csv files in aas_out/ (find_aas.py output) and franken_out/
- convert the aa data csv files to json
- convert the franken data csv files to json
- create the data dictionary and put json data in
- get fold data from SCOP and write the fold for each Viper pdb_id to the data dictionary 
- write RCSB data from families.xlsx and id_genus.txt to data dictionary
- write the gauge points from the point data
- rename some ViperDB genomes for simplicity and consistency purposes
- write aa data of full capsid to data dictionary (aas.json is made by count_aas.py)
- write atom count data (id_atoms.txt is made by analyze_aas.ipynb)
- compile data about the closest aa (& other aa within 5 Angstroms) for the closest point for the single top line in franken output, along with other aa data described in the README
- separate into discrete and continuous properties, change data types accordingly, and add the lists to dictionary
- create filters for sorting discrete variables

In [1]:
# convert all xl files to csv
import openpyxl
import csv
import json 
import sys
import re

from pathlib import Path
import os

Path("./aas_out/").mkdir(parents=True, exist_ok=True)
Path("./franken_out/").mkdir(parents=True, exist_ok=True)
for filename in os.listdir('xlfiles'):
    if '.xlsx' not in filename: continue
    wb = openpyxl.load_workbook('xlfiles/' + filename) 
    if 'full' in filename:
        for name in wb.sheetnames:
            sh = wb[name]
            v = filename.split('full_')[-1].split('.')[0]
            Path("./aas_out/" + v + '/').mkdir(parents=True, exist_ok=True)
            with open('aas_out/' + v + '/' + name + '.csv', 'w', newline="") as file_handle:
                csv_writer = csv.writer(file_handle)
                for row in sh.iter_rows(): # generator; was sh.rows
                    csv_writer.writerow([cell.value for cell in row])
    else:
        sh = wb.active 
        with open('franken_out/' + filename.split('.')[0] + '.csv', 'w', newline="") as file_handle:
            csv_writer = csv.writer(file_handle)
            for row in sh.iter_rows(): # generator; was sh.rows
                csv_writer.writerow([cell.value for cell in row])

In [2]:
#convert aa csv files to json
def aa_csv_to_json(csvFilePath, jsonFilePath):
    chain_dict = {}
    with open(csvFilePath, encoding='utf-8') as csvf: 
            chain_dict = {}
            x = [l for l in csvf]
            ixs = [m.start() for m in re.finditer('Chain', x[0])]
            chain_titles = [x[0][i:i+7] for i in ixs]
            for i, c in enumerate(chain_titles):
                c_buff = i*4
                for j in range(2, len(x)):
                    if c not in chain_dict.keys():
                        chain_dict[c] = []
                    c_dict = {}
                    c_dict['Point'] = x[j].split(',')[0]
                    c_dict['Distance'] = x[j].split(',')[c_buff + 1]
                    c_dict['Atom'] = x[j].split(',')[c_buff + 2]
                    c_dict['AARN'] = x[j].split(',')[c_buff + 3]
                    c_dict['Other AA'] = x[j].split(',')[c_buff + 4]
                    chain_dict[c].append(c_dict)
  
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(chain_dict, indent=4)
        jsonf.write(jsonString)


for filename in os.listdir('aas_out'):
    for csv_file in os.listdir('aas_out/' + filename + '/'):
        try:
            Path("aas_json/" + filename).mkdir(parents=True, exist_ok=True)
            csvFilePath = 'aas_out/' + filename + '/' + csv_file
            jsonFilePath = 'aas_json/' + filename + '/' + csv_file.split('.')[0] + '.json'
            aa_csv_to_json(csvFilePath, jsonFilePath)
        except Exception as e:
            print(csv_file)
            print(e)


In [3]:
# convert franken csv files to json
def franken_csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
        
from pathlib import Path
import os

Path("franken_json/").mkdir(parents=True, exist_ok=True)
for csv_file in os.listdir('franken_out'):
    try:
        filename =csv_file.split('.')[0]
        csvFilePath = 'franken_out/' + csv_file
        jsonFilePath = 'franken_json/' + filename + '.json'
        franken_csv_to_json(csvFilePath, jsonFilePath)
    except Exception as e:
        print(csv_file)
        print(e)

In [4]:
# put json data into dictionary
data = {'data': {}}
for v_dir in os.listdir('aas_json'):
    pa_dict = {}
    for json_file in os.listdir('aas_json/' + v_dir + '/'):
        json_path = 'aas_json/' + v_dir + '/' + json_file
        with open(json_path, 'r') as f:
            pa_data = json.load(f)
            pa_dict[json_file.split('_')[-1].split('.')[0]] = pa_data
    if v_dir not in data['data'].keys(): data['data'][v_dir] = {}
    data['data'][v_dir]['point_arrays'] = pa_dict
    
for v_dir in os.listdir('franken_json'):
    pa_dict = {}
    v = v_dir.split('.')[0]
    json_path = 'franken_json/' + v_dir
    with open(json_path, 'r') as f:
        franken_data = json.load(f)
    if v not in data['data'].keys(): data['data'][v] = {}
    data['data'][v]['points'] = franken_data

In [5]:
'''
get fold data for each virus from SCOP and write it to json
fold_trans.txt contains all SCOP codes which map to each fold as described in Nasir
this was done by finding the fold categories corresponding to folds in Nasir as they describe,
then getting the SCOP IDS which map to each of those categories
all_ids.txt contains each pdb_id on Viper (the code to create it is no longer working bc of API changes)
'''
# SCOP DOWNLOAD LINKS
#http://scop.mrc-lmb.cam.ac.uk/download

#SCOP domain definitions and classification.
#File: scop-cla-latest.txt
scop_url = 'data/scop.txt'

#SCOP node descriptions.
#File: scop-des-latest.txt
des_file = 'data/scop-des.txt'

#SCOP represented structures.
#File: scop-represented-structures-latest.txt
scop_repr = 'data/scop-repr.txt'
fold_dict = {}

with open('data/fold_trans.txt', 'r') as f:
    for l in f:
        fid = l.split()[0]
        ftrans = l.split(' ',1)[-1].replace('\n', '')
        fold_dict[fid] = ftrans

with open('data/all_ids.txt', 'r') as f:
    pdb_ids = [l.replace('\n', '').upper() for l in f]

try:
    pdb_ids.remove('1E6T')
    pdb_ids.append('2BU1')
    pdb_ids.remove('3IYU')
    pdb_ids.append('4V7Q')
except Exception as e:
    print(e)



with open(scop_url, 'r') as f:
    id_res = {l.split(' ')[1]:l.split(' ')[-1] for l in f if l[0] != '#'}
with open(scop_url, 'r') as f:
    uid_res = {l.split(' ')[0]:l.split(' ')[-1] for l in f if l[0] != '#'}
with open(des_file, 'r') as f:
    des_dict = {l.split(' ')[0]: l.split(' ', 1)[1] for l in f if l[0]!='#'}

id_fold = {}
for i in pdb_ids:
    if i in id_res:
        ires = id_res[i]
        for sid, trans in fold_dict.items():
            if sid in ires:
                id_fold[i] = trans

with open(scop_repr, 'r') as f:
    repr_lst = [l.replace('\n', '').split(' ') for l in f]

id_unident_uid = {}
for i in repr_lst:
    if i[0] not in id_unident_uid.keys():
        id_unident_uid[i[0]] = []
    id_unident_uid[i[0]].append(i[2])
    
for i in pdb_ids:
    if i.upper() not in id_fold and i.upper() in id_unident_uid.keys():
        for uid in id_unident_uid[i.upper()]:
            if uid in uid_res:
                res = uid_res[uid]
                for fld in fold_dict:
                    if fld in res:
                        id_fold[i] = fold_dict[fld]
                        continue
try:
    id_fold['1E6T'] = id_fold.pop('2BU1')
    id_fold['3IYU'] = id_fold.pop('4V7Q')
except Exception as e:
    print(e)
with open('data/id_fold.txt', 'w') as f:
    for k, v in id_fold.items():
        f.write(k.lower() + ' ' + v + '\n')

with open('id_fold.txt', 'r') as f:
    id_fold = {l.split()[0]: l.replace(l.split()[0], '')[1:].replace('\n','') for l in f}
for k,v in id_fold.items():
    if k not in data['data'].keys(): data['data'][k] = {}
    data['data'][k]['fold'] = v

In [6]:
#write RCSB data from families.xlsx to data dictionary
#The code used to make families.xlsx and id_genus.txt is no longer working bc of API changes but is in family_stats.ipynb
Path("./fams/").mkdir(parents=True, exist_ok=True)
Path("./json_fams/").mkdir(parents=True, exist_ok=True)
filename = 'data/families.xlsx'
wb = openpyxl.load_workbook(filename) 
for name in wb.sheetnames:
    sh = wb[name]
    with open('fams/' + name + '.csv', 'w', newline="") as file_handle:
        csv_writer = csv.writer(file_handle)
        for row in sh.iter_rows(): # generator; was sh.rows
            csv_writer.writerow([cell.value for cell in row])

for filename in os.listdir('fams'):
    if '.csv' not in filename: continue
    fam = filename.split('.')[0]
    csvFilePath = 'fams/' + fam + '.csv'
    jsonFilePath = 'json_fams/' + fam + '.json'
    def csv_to_json(csvFilePath, jsonFilePath):
        jsonArray = []

        #read csv file
        with open(csvFilePath, encoding='utf-8') as csvf: 
            #load csv file data using csv library's dictionary reader
            csvReader = csv.DictReader(csvf) 

            #convert each csv row into python dict
            for row in csvReader: 
                #add this python dict to json array
                if ',' in row['genome']: row['genome'] = row['genome'].replace(',', ';')
                jsonArray.append(row)
        
        with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
            jsonString = json.dumps(jsonArray, indent=4)
            jsonf.write(jsonString)

    csv_to_json(csvFilePath, jsonFilePath)

for filename in os.listdir('json_fams'):
    fam = filename.split('.')[0]
    fpath = 'json_fams/' + filename
    with open(fpath, 'r') as f:
        fam_json = json.load(f)
        for el in fam_json:
            v = el['id']
            if v not in data['data']: data['data'][v] = {}
            for k in el.keys():
                if k == 'id': continue
                data['data'][v][k] = el[k]
            data['data'][v]['family'] = fam
            data['data'][v]['average_radius'] = data['data'][v].pop('radius')
            
with open('data/id_genus.txt', 'r') as f:
    id_genus = {l.split()[0]:' '.join(l.split()[1:]) for l in f}
for v in data['data']:
    if v in id_genus:
        data['data'][v]['genus'] = id_genus[v]


In [5]:
#write resolution data from ViperDB to data dictionary
#the code used to make res.txt (containing resolution data) is no longer working

with open("data/res.txt", 'r') as f:
    id_res = {l.split()[0]: l.split()[1] for l in f}
for i in id_res:
    data['data'][i]['resolution'] = id_res[i]

In [7]:
#write the gauge points from the point data
gp_dict = {k:data['data'][k]['points'][0]['GP'] for k in data['data'].keys() if 'points' in data['data'][k].keys()}

for k, v in gp_dict.items():
    data['data'][k]['gauge_point'] = v

data['gauge_points'] = {k:data['data'][k].pop('points') for k in data['data'] if 'points' in data['data'][k]}

In [86]:
#rename some ViperDB genomes for simplicity and consistency purposes
genome_dict = {}
genome_dict['dsRNA viruses'] = 'dsRNA'
genome_dict['ssDNA viruses'] = 'ssDNA'
genome_dict['Circular DNA'] = 'circular DNA'
genome_dict['ssRNA viruses'] = 'ssRNA'
genome_dict['sRNA'] = 'ssRNA'
genome_dict['ssRNA positive-strand viruses; no DNA stage'] = 'ssRNA positive-strand viruses'
genome_dict['ssRNA positive-strand viruse; no DNA stage'] = 'ssRNA positive-strand viruses'
genome_dict['dsDNA viruses; no RNA stage'] = 'dsDNA'
genome_dict['ssRNA-RT'] = 'ssRNA'
genome_dict['ssRNA positive-strand viruses'] = 'ssRNA'
genome_dict['ssRNA viruses; no DNA stage'] = 'ssRNA'
genome_dict['ssRNA-RT'] = 'ssRNA'
genome_dict['circular DNA'] = 'dsDNA'

for v in data['data'].keys():
    if 'genome' not in data['data'][v].keys(): continue
    if isinstance(data['data'][v]['genome'], list): 
        if data['data'][v]['genome'] == ['dsDNA']:
            data['data'][v]['genome'] = 'dsDNA'
    elif data['data'][v]['genome'] in genome_dict:
        data['data'][v]['genome'] = genome_dict[data['data'][v]['genome']]

In [9]:
# write aa data of full capsid to data dictionary (aas.json is made by count_aas.py)
with open('aas.json', 'r') as f:
    aa_dict = json.load(f)

nw_aas = {}

for v in aa_dict:
    nw_aas[v[-4:]] = aa_dict[v]

data['aas'] = {}

for k, v in nw_aas.items():
    if k not in data['data'].keys(): data['data'][k] = {}
    data['aas'][k] = v
    data['data'][k]['most_common_aa'] = max(v, key=v.get)

In [3]:
# write atoms data of capsid (id_atoms.txt is made by analyze_aas.ipynb)
# excluding data if the capsid only has c alpha atoms (also in analyze_aas)
with open('data/id_atoms.txt', 'r') as f:
    id_atoms = {l.split()[0]:l.split()[1].replace('\n', '') for l in f}
with open('data/only_ca.txt', 'r') as f:
    ca_ids = [l.replace('\n', '') for l in f]

for i in data['data']:
    if 'atoms' in data['data'][i]:
        del data['data'][i]['atoms']
for i in id_atoms:
    if i not in ca_ids:
        data['data'][i]['atoms'] = int(id_atoms[i])

In [10]:
#compile data about the closest aa (& other aa within 5 Angstroms) for the closest point for each of the top 5 PAs in franken output
import openpyxl
from os import listdir
from os.path import isfile, join
mypath = '../getvirons/xlfiles/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

from plot_util import *

def letters(s):
    valids = []
    for character in s:
        if character.isalpha():
            valids.append(character)
    return ''.join(valids)
data['all_pas_closest'] = {}
data['close_aas'] = {}
for f in onlyfiles:
    if 'full' not in f or '.xlsx' not in f: continue
    xldir = mypath + f
    wb = openpyxl.load_workbook(xldir)
    v = f.split('_', 1)[-1].split('.')[0]
    dclosest_aas = []
    for sheetname in wb.sheetnames:
        sh = wb[sheetname]
        vitems = []
        for row in sh.iter_rows():
                vitems.append([cell.value for cell in row])
        aas = []
        cnt = 0
        ds_aas = {}
        for i in range(len(vitems[0])):
            for j in range(2, len(vitems)):
                if vitems[j][i] is not None and '.' in str(vitems[j][i]):
                    d = vitems[j][i]
                    ds_aas[d] = vitems[j][i:i+4]
                    if d < 5:
                        aas.append(letters(vitems[j][i+2]))
                        if vitems[j][i+3] != 'N/A': aas.append(letters(vitems[j][i+3]))
        closest_aa = letters(ds_aas[min(ds_aas.keys())][2])
        other_aa = letters(ds_aas[min(ds_aas.keys())][3])
        dclosest_aas.append([closest_aa, other_aa])
    data['all_pas_closest'][v] = dclosest_aas

In [11]:
#compile data about the closest aa (& other aa within 5 Angstroms) for the closest point for the single top line in franken output, along with other aa data described in the README
mypath = '../getvirons/xlfiles/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
idpath = '../getvirons/xlfiles_prechain/'
idpafiles = [f for f in listdir(idpath) if isfile(join(idpath, f))]
id_pa = {}
for f in idpafiles:
    try:
        if 'full' in f: continue
        v = f.split('.xlsx')[0]
        xldir = idpath + f
        wb = openpyxl.load_workbook(xldir)
        name = wb.sheetnames[0]
        sh = wb[name]
        r = sh[2]
        closest_pa = r[0].value
        id_pa[v] = closest_pa
    except Exception as e:
        print(v)

with open('id_pa.txt', 'w') as f:
    for i in id_pa:
        f.write(i + ' ' + str(id_pa[i])+ '\n')

data['close_aas'] = {}
for f in onlyfiles:
    if 'full' not in f or '.xlsx' not in f: continue
    xldir = mypath + f
    wb = openpyxl.load_workbook(xldir)
    v = f.split('_', 1)[-1].split('.')[0]
    pa = id_pa[v]
    pa_sh = f"{v}_pa_{pa}"
    resdict = []
    sh = wb[pa_sh]
    vitems = []
    for row in sh.iter_rows():
            vitems.append([cell.value for cell in row])
    aas = []
    ds_aas = {}
    gp_aas = {}
    for j in range(2, len(vitems)):
        for i in range(len(vitems[0])):
            if vitems[j][i] is not None and '.' in str(vitems[j][i]):
                d = vitems[j][i]
                ds_aas[d] = vitems[j][i:i+4]
                if j == 2:
                    gp_aas[d] = ds_aas[d]
                if d < 5:
                    aas.append(letters(vitems[j][i+2]))
                    if vitems[j][i+3] != 'N/A': aas.append(letters(vitems[j][i+3]))
    resdict.extend(aas)
    closest_gp_aa = letters(gp_aas[min(gp_aas.keys())][2])
    other_gp_aa = letters(gp_aas[min(gp_aas.keys())][3])
    closest_aa = letters(ds_aas[min(ds_aas.keys())][2])
    other_aa = letters(ds_aas[min(ds_aas.keys())][3])
    data['data'][v]['closest_gp_aa'] = closest_gp_aa
    data['data'][v]['other_gp_aa'] = other_gp_aa
    data['data'][v]['closest_aa'] = closest_aa
    data['data'][v]['other_aa'] = other_aa
    cres = countlist(resdict)
    data['close_aas'][v] = cres
    try:
        data['data'][v]['common_gauge_aa'] = max(cres, key=cres.get)
    except Exception as e:
        #print(v)
        pass
for f in onlyfiles:
    if 'full' not in f: continue
    xldir = mypath + f
    wb = openpyxl.load_workbook(xldir)
    v = f.split('_', 1)[-1].split('.')[0]
    resdict = []
    for name in wb.sheetnames:
            sh = wb[name]
            vitems = []
            for row in sh.iter_rows():
                    vitems.append([cell.value for cell in row])
            aas = []
            cnt = 0
            for i in range(len(vitems[0])):
                for j in range(2, len(vitems)):
                    if vitems[j][i] is not None and '.' in str(vitems[j][i]):
                        d = vitems[j][i]
                        if d < 5:
                            aas.append(letters(vitems[j][i+2]))
                            if vitems[j][i+3] != 'N/A': aas.append(letters(vitems[j][i+3]))
            resdict.extend(aas)
    cres = countlist(resdict)
    data['close_aas'][v] = cres
    data['data'][v]['common_gauge_aa'] = max(cres, key=cres.get)

In [7]:
#separate into discrete and continuous properties, change data types accordingly, and add the lists to dictionary
for v in data['data']:
    if 'point_arrays' in data['data'][v]: del data['data'][v]['point_arrays']

if 'gauge_points' in data: del data['gauge_points']
data['data']['7wqp']['genome'] = 'NA'
properties = {}
properties['discrete'] = []
properties['continuous'] = []

def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)

var_dict = {}
for k in data['data']:
    if 'tnum' in data['data'][k].keys(): del data['data'][k]['tnum']
    for v in data['data'][k]:
        if v not in var_dict.keys(): var_dict[v] = 'continuous'
        if not str(data['data'][k][v]).replace('.','',1).isdigit() and len(str(data['data'][k][v])) > 0: 
            var_dict[v] = 'discrete'

conts = [k for k,v in var_dict.items() if v=='continuous']
discs = [k for k,v in var_dict.items() if v=='discrete']
discs.append('gauge_point')
conts.remove('gauge_point')
properties['continuous'] = conts
properties['discrete'] = discs

for virus in data['data']:
    for c in conts:
        if c in data['data'][virus].keys():
            item = str(data['data'][virus][c])
            if len(item) == 0: del data['data'][virus][c]
            else: data['data'][virus][c] = num(item)

data['properties'] = properties

In [8]:
#create filters for sorting by discrete variables
import numpy as np
ks = {}
for v in data['data']:
    for k in data['data'][v]:
        if k not in data['properties']['discrete']: continue
        if k not in ks: ks[k] = []
        if data['data'][v][k] == '': data['data'][v][k] = 'NA'
        if str(data['data'][v][k]) not in ks[k]: ks[k].append(str(data['data'][v][k]))
for f in ks:
    ks[f] = sorted(ks[f])
ks['gauge_point'] = [str(s) for s in sorted([int(p) for p in ks['gauge_point']])]
trank = [int(''.join([s for s in t if s.isdigit()])) if len([s for s in t if s.isdigit()]) > 0 else 999 for t in ks['tnumber']]
tsort = np.argsort(trank)
ts = [0]*20
for i in range(len(tsort)):
    el = ks['tnumber'][tsort[i]]
    ts[i] = el
ks['tnumber'] = ts
data['filters'] = ks

In [5]:
with open('data/db.json', 'w') as f:
    json_str = json.dumps(data, indent=4)
    f.write(json_str)